## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-20-06-01-11 +0000,bbc,Grey's Anatomy star Eric Dane dies at 53 after...,https://www.bbc.com/news/articles/c78j01n775ko...
1,2026-02-20-06-00-17 +0000,nypost,"Benjamin Shroats, former coach of US Olympic f...",https://nypost.com/2026/02/20/us-news/amber-gl...
2,2026-02-20-05-59-56 +0000,nypost,Tahoe avalanche victims allegedly took most da...,https://nypost.com/2026/02/20/us-news/lake-tah...
3,2026-02-20-05-50-32 +0000,nypost,Family says law enforcement can ‘do better’ af...,https://nypost.com/2026/02/20/us-news/florida-...
4,2026-02-20-05-47-32 +0000,nyt,Why Is Trump Dumping East Wing Rubble in a Pub...,https://www.nytimes.com/2026/02/19/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2591/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
44,trump,58
110,epstein,31
178,andrew,31
177,prince,27
54,iran,25


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
198,2026-02-19-20-12-03 +0000,nypost,Trump expresses sympathy for the royal family ...,https://nypost.com/2026/02/19/us-news/trump-ex...,187
125,2026-02-19-23-00-14 +0000,nyt,British Police Arrest Former Prince Andrew Ami...,https://www.nytimes.com/2026/02/19/world/europ...,176
191,2026-02-19-20-28-03 +0000,nyt,The police are investigating former Prince And...,https://www.nytimes.com/live/2026/02/19/world/...,160
141,2026-02-19-22-28-26 +0000,nypost,Bombshell Jeffrey Epstein files that led to ex...,https://nypost.com/2026/02/19/world-news/these...,156
212,2026-02-19-19-31-44 +0000,nypost,Ex-Prince Andrew looks aghast in first photos ...,https://nypost.com/2026/02/19/world-news/ex-pr...,153


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
198,187,2026-02-19-20-12-03 +0000,nypost,Trump expresses sympathy for the royal family ...,https://nypost.com/2026/02/19/us-news/trump-ex...
390,96,2026-02-19-07-49-29 +0000,nypost,Former South Korean President Yoon sentenced t...,https://nypost.com/2026/02/19/world-news/forme...
101,73,2026-02-20-00-00-00 +0000,wsj,The husband of embattled Labor Secretary Lori ...,https://www.wsj.com/politics/policy/labor-secr...
172,68,2026-02-19-21-06-00 +0000,nypost,UK condemns ‘totally unjustifiable’ 10-year es...,https://nypost.com/2026/02/19/world-news/uk-co...
10,54,2026-02-20-05-04-48 +0000,nypost,Les Wexner’s attorney caught on hot mic saying...,https://nypost.com/2026/02/20/us-news/les-wexn...
16,52,2026-02-20-04-44-31 +0000,nypost,New Tahoe ski death as student found on black ...,https://nypost.com/2026/02/19/us-news/new-taho...
196,52,2026-02-19-20-13-08 +0000,nyt,House Democrats view Mountbatten-Windsor’s arr...,https://www.nytimes.com/live/2026/02/19/world/...
387,49,2026-02-19-08-13-25 +0000,bbc,Bill Gates pulls out of India's AI summit over...,https://www.bbc.com/news/articles/c309qv9zglno...
311,45,2026-02-19-13-48-20 +0000,nypost,Middle Eastern funds are ready to invest tens ...,https://nypost.com/2026/02/19/business/jpmorga...
279,43,2026-02-19-15-39-00 +0000,wsj,"The U.S. trade deficit jumped in December, the...",https://www.wsj.com/economy/trade/u-s-trade-de...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
